# Quick Start: Evaluación de Base de Datos en 3 Pasos

Este notebook permite evaluar una base de datos de manera rápida y directa.

## Requisitos previos

1. Dependencias instaladas: `pip install -r requirements_fase3.txt`
2. Modelo spaCy descargado: `python -m spacy download es_core_news_sm`
3. Configuración `.env` con `OPENAI_API_KEY` (opcional para diagnóstico IA)

---

## Paso 1: Setup e Imports

In [1]:
import pandas as pd
import numpy as np
import sys
import json
from pathlib import Path
from datetime import datetime

# Agregar ruta del proyecto
sys.path.append('..')

from fase3_evaluator.integration import run_parallel_analysis, get_analysis_summary
from fase3_evaluator.agent import generate_diagnosis
from config import settings

print("Sistema listo")
print(f"OpenAI configurado: {'Si' if settings.OPENAI_API_KEY else 'No (solo analisis tecnico)'}")

Sistema listo
OpenAI configurado: Si


## Paso 1b: Función Helper para Conversión de Tipos NumPy

Esta función es necesaria para evitar errores de serialización JSON con tipos de NumPy.

In [2]:
def convert_numpy_types(obj):
    """
    Convierte tipos NumPy/Pandas a tipos nativos Python para JSON serialization.
    
    Args:
        obj: Objeto a convertir
        
    Returns:
        Objeto convertido a tipos nativos Python
    """
    if isinstance(obj, dict):
        return {key: convert_numpy_types(value) for key, value in obj.items()}
    elif isinstance(obj, list):
        return [convert_numpy_types(item) for item in obj]
    elif isinstance(obj, (np.integer, np.int64, np.int32, np.int16, np.int8)):
        return int(obj)
    elif isinstance(obj, (np.floating, np.float64, np.float32, np.float16)):
        return float(obj)
    elif isinstance(obj, (np.bool_, bool)):
        return bool(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif pd.isna(obj):
        return None
    else:
        return obj

print("Funcion de conversion cargada")

Funcion de conversion cargada


## Paso 2: Carga de Datos

### Opción A: Archivo CSV propio

In [3]:
# OPCION A: Carga de archivo CSV propio
# Modificar esta ruta con la ubicación del archivo
ruta_archivo = "../data/mi_archivo.csv"

# Descomentar la siguiente línea para cargar el archivo:
# df = pd.read_csv(ruta_archivo)
# df.head()

### Opción B: Dataset de Ejemplo (para pruebas)

In [4]:
# OPCION B: Dataset de ejemplo para probar el sistema
df = pd.DataFrame({
    'id': range(1, 51),
    'fecha_evento': pd.date_range('2024-01-01', periods=50, freq='W'),
    'edad': [25, 30, None, 45, 52, 38, 60, 28, 35, 42] * 5,
    'sexo': ['M', 'F', 'M', 'M', 'F'] * 10,
    'metodo': ['ahorcamiento', 'intoxicacion', None, 'arma de fuego', 'ahorcamiento'] * 10,
    'municipio': ['Madrid', 'Valencia', 'Barcelona', 'Sevilla', 'Bilbao'] * 10,
    'latitud': [40.4168, 39.4699, 41.3851, 37.3891, 43.2627] * 10,
    'longitud': [-3.7038, -0.3763, 2.1734, -5.9845, -2.9253] * 10,
    'tipo_evento': ['consumado', 'intento', 'consumado', 'intento', 'consumado'] * 10
})

print(f"Dataset cargado: {len(df)} registros, {len(df.columns)} columnas")
df.head()

Dataset cargado: 50 registros, 9 columnas


,id,fecha_evento,edad,sexo,metodo,municipio,latitud,longitud,tipo_evento
0,1,2024-01-07,25.0,M,ahorcamiento,Madrid,40.4168,-3.7038,consumado
1,2,2024-01-14,30.0,F,intoxicacion,Valencia,39.4699,-0.3763,intento
2,3,2024-01-21,NaN,M,None,Barcelona,41.3851,2.1734,consumado
3,4,2024-01-28,45.0,M,arma de fuego,Sevilla,37.3891,-5.9845,intento
4,5,2024-02-04,52.0,F,ahorcamiento,Bilbao,43.2627,-2.9253,consumado


## Paso 3: Ejecución del Análisis Completo

### 3.1 Análisis Técnico (6 dimensiones en paralelo)

In [5]:
print("Ejecutando analisis tecnico...\n")

# Ejecutar análisis con anonimización automática si se detecta PII
consolidated_json, df_anonymized, anonymization_report = run_parallel_analysis(
    df,
    filename="mi_dataset.csv",
    auto_anonymize=True
)

print("\nAnalisis completado")

INFO:fase3_evaluator.integration.orchestrator:Iniciando análisis paralelo de 'mi_dataset.csv' con 50 registros y 9 columnas
INFO:fase3_evaluator.integration.orchestrator:Ejecutando analizadores en paralelo...
INFO:fase3_evaluator.integration.orchestrator:✓ geospatial completado
INFO:fase3_evaluator.integration.orchestrator:✓ completeness completado
INFO:fase3_evaluator.integration.orchestrator:✓ typology completado
INFO:fase3_evaluator.integration.orchestrator:✓ anonymization completado
INFO:fase3_evaluator.integration.orchestrator:✓ semantic completado
INFO:fase3_evaluator.integration.orchestrator:✓ ml_readiness completado
INFO:fase3_evaluator.integration.orchestrator:Anonimización completada y validada
INFO:fase3_evaluator.integration.orchestrator:Consolidando resultados...
ml.model_suggestions.0.models
  Input should be a valid string [type=string_type, input_value=['Logistic Regression', '...ndom Forest', 'XGBoost'], input_type=list]
    For further information visit https://errors

Ejecutando analisis tecnico...


Analisis completado


### 3.2 Resumen Ejecutivo

In [6]:
# Generar resumen ejecutivo
summary = get_analysis_summary(consolidated_json)

print("=" * 70)
print("RESUMEN EJECUTIVO")
print("=" * 70)

# Información del dataset
print(f"\nDataset:")
print(f"   {summary['dataset']['rows']:,} registros x {summary['dataset']['columns']} columnas")

# Scores de calidad
print(f"\nCalidad de los Datos:")
for metric, score in summary['scores'].items():
    bar = "#" * int(score/5) + "." * (20 - int(score/5))
    status = "OK" if score >= 70 else "REVISAR" if score >= 50 else "CRITICO"
    print(f"   {metric.capitalize():15s} [{bar}] {score:.0f}/100 ({status})")

# Score general
overall = summary['overall_score']
bar_overall = "#" * int(overall/5) + "." * (20 - int(overall/5))
print(f"\n   {'SCORE GENERAL':15s} [{bar_overall}] {overall:.0f}/100")

# Issues críticos
print(f"\nIssues Criticos:")
has_issues = False
if summary['critical_issues']['pii_detected']:
    print(f"   [!] PII detectada: {summary['critical_issues']['pii_risk_level'].upper()}")
    if anonymization_report:
        print(f"       Anonimizada automaticamente")
    has_issues = True
if summary['critical_issues']['completitud_critica']:
    print(f"   [!] Completitud critica en campos clave")
    has_issues = True
if summary['critical_issues']['leakage_risks'] > 0:
    print(f"   [!] {summary['critical_issues']['leakage_risks']} riesgos de data leakage")
    has_issues = True
if not has_issues:
    print(f"   No se detectaron issues criticos")

# Capacidades analíticas
print(f"\nCapacidades Analiticas:")
if summary['capabilities']['geocodable']:
    print(f"   [+] Analisis geoespacial y mapas")
if summary['capabilities']['clustering_feasible']:
    print(f"   [+] Clustering (deteccion de patrones espaciales)")
if summary['capabilities']['ml_viable']:
    print(f"   [+] Machine Learning (modelos predictivos)")

print("\n" + "=" * 70)

RESUMEN EJECUTIVO

Dataset:
   50 registros x 9 columnas

Calidad de los Datos:
   Completitud     [###################.] 97/100 (OK)
   Tipologia       [###################.] 96/100 (OK)
   Semantica       [####################] 100/100 (OK)
   Geoespacial     [####################] 100/100 (OK)
   Ml_viabilidad   [##########..........] 50/100 (REVISAR)

   SCORE GENERAL   [#################...] 88/100

Issues Criticos:
   [!] PII detectada: MODERADO
       Anonimizada automaticamente

Capacidades Analiticas:
   [+] Analisis geoespacial y mapas
   [+] Clustering (deteccion de patrones espaciales)



### 3.3 Diagnóstico con IA (Opcional)

Requiere `OPENAI_API_KEY` configurada en `.env`

In [7]:
if settings.OPENAI_API_KEY:
    print("Generando diagnostico interpretativo con IA...\n")
    
    try:
        diagnosis_result = generate_diagnosis(
            consolidated_json,
            df_anonymized,
            include_sample=True
        )
        
        print("=" * 70)
        print("DIAGNOSTICO GENERADO POR IA")
        print("=" * 70)
        print(f"Modelo: {diagnosis_result['model_used']} | Tokens: {diagnosis_result['tokens_used']}")
        print("=" * 70)
        print("\n" + diagnosis_result['diagnosis'])
        
    except Exception as e:
        print(f"Error al generar diagnostico: {str(e)}")
else:
    print("OpenAI no configurado - solo analisis tecnico disponible")
    print("Para habilitar diagnostico IA: configurar OPENAI_API_KEY en .env")

INFO:fase3_evaluator.agent.interpreter:Generando diagnóstico con OpenAI...


Generando diagnostico interpretativo con IA...



INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:fase3_evaluator.agent.interpreter:Diagnóstico generado exitosamente


DIAGNOSTICO GENERADO POR IA
Modelo: gpt-4o | Tokens: 6224

## 1. DIAGNÓSTICO GENERAL

El dataset analizado contiene 50 registros con 9 columnas, cubriendo eventos relacionados con el suicidio. La calidad general de los datos es excelente, con un puntaje de completitud del 96.67%. Sin embargo, se identificaron algunas áreas críticas que requieren atención, como la inconsistencia en la tipología de datos en la columna "sexo" y la presencia de información potencialmente identificable (PII) en la columna "id".

Los hallazgos más importantes incluyen: 
1. Una inconsistencia significativa en la columna "sexo", donde el 40% de los valores no coinciden con el tipo de dato esperado.
2. La columna "metodo" presenta un 20% de datos faltantes, lo que limita el análisis detallado de los métodos utilizados.
3. La presencia de PII en la columna "id", lo que implica un riesgo moderado de re-identificación.

## 2. ANÁLISIS QUE SÍ SE PUEDEN REALIZAR

✅ **Análisis Descriptivo de Distribuciones**
📊 Permit

## Paso 4: Exploración de Resultados Detallados (Opcional)

### 4.1 Análisis de Completitud

In [8]:
completeness = consolidated_json['completitud']

print("ANALISIS DE COMPLETITUD")
print(f"Score: {completeness['evaluation']['score']:.1f}/100")
print(f"Valores faltantes: {completeness['summary']['missing_percentage']:.1f}%")

if completeness['critical_fields_missing']:
    print(f"\nCampos criticos faltantes: {', '.join(completeness['critical_fields_missing'])}")

print("\nTop columnas con mas valores faltantes:")
for col_info in completeness['top_missing_columns'][:5]:
    print(f"  - {col_info['column']}: {col_info['missing_rate']*100:.1f}%")

ANALISIS DE COMPLETITUD
Score: 96.7/100
Valores faltantes: 3.3%

Top columnas con mas valores faltantes:
  - metodo: 20.0%
  - edad: 10.0%
  - id: 0.0%
  - fecha_evento: 0.0%
  - sexo: 0.0%


### 4.2 Análisis ML Readiness

In [9]:
ml = consolidated_json['ml']

print("ANALISIS ML READINESS")
print(f"Viable: {'Si' if ml['ml_viability']['viable'] else 'No'}")
print(f"Confianza: {ml['ml_viability']['confidence']}")
print(f"Score: {ml['ml_viability']['score']:.1f}/100")

print(f"\nTarget detectado: {ml['target_column']}")
print(f"Features usables: {ml['features_analysis']['usable_features']}")

if ml['model_suggestions']:
    print("\nModelos sugeridos:")
    for suggestion in ml['model_suggestions'][:3]:
        print(f"\n  {suggestion['task']}:")
        print(f"    Modelos: {', '.join(suggestion['models'])}")
        print(f"    Razon: {suggestion['reason']}")

ANALISIS ML READINESS
Viable: No
Confianza: nula
Score: 50.0/100

Target detectado: tipo_evento
Features usables: 6

Modelos sugeridos:

  clasificacion_binaria:
    Modelos: Logistic Regression, Random Forest, XGBoost
    Razon: Dataset pequeño - modelos interpretables recomendados


### 4.3 Análisis de Privacidad

In [10]:
anon = consolidated_json['anonimizacion']

print("ANALISIS DE PRIVACIDAD Y ANONIMIZACION")
print(f"PII detectada: {'Si' if anon['summary']['pii_detected'] else 'No'}")

if anon['summary']['pii_detected']:
    print(f"Risk score: {anon['summary']['risk_score']:.1f}/10")
    print(f"Nivel: {anon['summary']['risk_level'].upper()}")
    
    print("\nEntidades detectadas:")
    for entity in anon['entities_found'][:5]:
        print(f"  - {entity['type']} en '{entity['column']}': {entity['count']} instancias")
    
    if anonymization_report:
        print("\nAnonimizacion aplicada:")
        for col, method in list(anonymization_report['transformation_details'].items())[:5]:
            print(f"  - {col}: {method}")
else:
    print("No se detecto informacion personal identificable")

ANALISIS DE PRIVACIDAD Y ANONIMIZACION
PII detectada: Si
Risk score: 3.3/10
Nivel: MODERADO

Entidades detectadas:
  - ID_NUMBER en 'id': 50 instancias

Anonimizacion aplicada:
  - id: removed


## Paso 5: Exportación de Resultados

In [11]:
# Crear carpeta de outputs
output_dir = Path('../data/outputs')
output_dir.mkdir(parents=True, exist_ok=True)

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# 1. JSON completo - Convertir tipos NumPy antes de guardar
json_path = output_dir / f'analisis_{timestamp}.json'
consolidated_json_clean = convert_numpy_types(consolidated_json)
with open(json_path, 'w', encoding='utf-8') as f:
    json.dump(consolidated_json_clean, f, indent=2, ensure_ascii=False)
print(f"JSON guardado: {json_path}")

# 2. Datos anonimizados
csv_path = output_dir / f'datos_anonimizados_{timestamp}.csv'
df_anonymized.to_csv(csv_path, index=False)
print(f"CSV guardado: {csv_path}")

# 3. Diagnóstico (si existe)
if 'diagnosis_result' in locals():
    md_path = output_dir / f'diagnostico_{timestamp}.md'
    with open(md_path, 'w', encoding='utf-8') as f:
        f.write(f"# Diagnostico Automatico\n")
        f.write(f"Generado: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
        f.write(diagnosis_result['diagnosis'])
    print(f"Diagnostico guardado: {md_path}")

print(f"\nTodos los archivos guardados en: {output_dir.absolute()}")

JSON guardado: ../data/outputs/analisis_20251125_122639.json
CSV guardado: ../data/outputs/datos_anonimizados_20251125_122639.csv
Diagnostico guardado: ../data/outputs/diagnostico_20251125_122639.md

Todos los archivos guardados en: /Users/reinerfuentesferrada/ONLINE_DS_THEBRIDGE_Rei/Proyecto ML/cuidar-ia/cuidar_ia_fase3 4/notebooks/../data/outputs


---

## Análisis Completado

He evaluado la base de datos completa en pocos minutos.

### Próximos pasos:

1. **Notebooks avanzados:**
   - `01_test_analizadores.ipynb` - Prueba individual de cada analizador
   - `02_flujo_completo.ipynb` - Flujo detallado con explicaciones

2. **Personalización:**
   - Ajustar umbrales en `config/settings.py`
   - Personalizar prompts en `fase3_evaluator/agent/prompts.py`

3. **Integración:**
   - Usar en Streamlit: `pages/5_Evaluador_Bases_Datos.py`
   - Integrar en scripts propios

### Soporte:
- Email: reinefuentes7@gmail.com
- Documentación: `../README.md`
- Instalación: `../docs/INSTALACION.md`